<a href="https://colab.research.google.com/github/antbartash/australian_rain/blob/main/MicrosoftNNI/SimulatedAnnealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Microsoft NNI on Google Colab / Kaggle

In [1]:
# install NNI and dependencies
! pip install nni==2.9
! pip install filelock==3.10
! pip install ConfigSpace

# download ngrok and unzip it
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 18.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 7.3 MB/s eta 0:00:00
  Obtaining dependency information for json-tricks>=3.15.5 from https://files.pythonhosted.org/packages/ae/fd/e3edcf827e7f9c17c5ea1a192841dcfb1dd575a7518c25c5cadd921625b1/json_tricks-3.17.3-py2.py3-none-any.whl.metadata
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.2.7
    Uninstalling hyperopt-0.2.7:
      Successfully uninstalled hyperopt-0.2.7
  Attempting uninstall: filelock
    Found existing installation: filelock 3.12.2
    Uninstalling filelock-3.12.2:
      Successfully uninstalled filelock-3.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.24.3 which is incompatible.
pymc3 3.11.5 req

In [2]:
! ./ngrok authtoken 2ZqSaEGuFWdJVe4YADpDuGtZ1R9_2L3tHZaAHSSavE6N96jo4 # YOUR_AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
# clone the github repo to get the model.py file
! git clone https://github.com/antbartash/australian_rain.git

Cloning into 'australian_rain'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 273 (delta 94), reused 105 (delta 56), pack-reused 101
Receiving objects: 100% (273/273), 8.75 MiB | 18.71 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [4]:
from nni.experiment import Experiment

In [5]:
hyperparameter_space = {
    # 'n_estimators': {'_type': 'randint', '_value': [100, 2000]}, # using n_estimators as the trial_budget
    # 'learning_rate': {'_type': 'loguniform', '_value': [1e-6, 0.5]}, # using CatBoost LR estimation
    'depth': {'_type': 'randint', '_value': [1, 10]},
    'l2_leaf_reg': {'_type': 'uniform', '_value': [0.0, 100.0]},
    # 'random_strength': {'_type': 'uniform', '_value': [0.0, 100.0]}, # CPU only
    'bagging_temperature': {'_type': 'uniform', '_value': [0.0, 100.0]},
    'grow_policy': {'_type': 'choice', '_value': ['SymmetricTree', 'Depthwise']},
    'scale_pos_weight': {'_type': 'uniform', '_value': [1, 4]}
}

In [6]:
experiment = Experiment('local')
experiment.config.experiment_name = 'CatBoost_NNI_BOHB'
experiment.config.tuner.name = 'BOHB'
experiment.config.tuner.class_args = {
    'optimize_mode': 'maximize',
    'min_budget': 1,
    'max_budget': 81, # default 27
    'eta': 3,
    'min_points_in_model': 8, # number of observations to start building a KDE
    'top_n_percent': 15, # used in multivariate TPE
    'num_samples': 64, # number of samples to optimize EI (default 64)
    'random_fraction': 0.33
}
experiment.config.experiment_working_directory = '/kaggle/working/'
experiment.config.trial_command = 'python3 /kaggle/working/australian_rain/MicrosoftNNI/model_GPU_HB.py'
experiment.config.trial_code_directory = '.'
experiment.config.search_space = hyperparameter_space
experiment.config.max_trial_number = 250
experiment.config.trial_concurrency = 1
experiment.config.max_experiment_duration = '2h'

In [7]:
get_ipython().system_raw('./ngrok http 5000 &')

In [8]:
! curl -s http://localhost:4040/api/tunnels # don't change the port number 4040

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://192b-34-136-217-202.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://192b-34-136-217-202.ngrok-free.app","proto":"http","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [9]:
experiment.run(5000, wait_completion=True, debug=False)

[2023-12-30 13:27:38] Creating experiment, Experiment ID: chfpozue
[2023-12-30 13:27:38] Starting web server...
[2023-12-30 13:27:39] Setting up...
[2023-12-30 13:27:39] Web portal URLs: http://127.0.0.1:5000 http://172.19.2.2:5000


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/statsmodels/nonparametric/kernels.py:62: RuntimeWarning: divide by zero encountered in divide
  kernel_value = np.ones(Xi.size) * h / (num_levels - 1)


In [11]:
best_trial = sorted(experiment.export_data(), key=lambda x: x.value, reverse=True)[0]
print(f"Best trial: {best_trial.parameter}")
print(f"Best score (AUC): {best_trial.value}")

Best trial: {'bagging_temperature': 0.9694377585891409, 'depth': 7, 'grow_policy': 'SymmetricTree', 'l2_leaf_reg': 74.39289705443491, 'scale_pos_weight': 1.8238007491880892, 'TRIAL_BUDGET': 9}
Best score (AUC): 0.8964027898324588


In [12]:
# zip NNI logs
!zip -r BOHBLogs.zip /kaggle/working/chfpozue

  adding: kaggle/working/chfpozue/ (stored 0%)
  adding: kaggle/working/chfpozue/db/ (stored 0%)
  adding: kaggle/working/chfpozue/db/nni.sqlite (deflated 96%)
  adding: kaggle/working/chfpozue/log/ (stored 0%)
  adding: kaggle/working/chfpozue/log/nnimanager.log (deflated 92%)
  adding: kaggle/working/chfpozue/log/experiment.log (deflated 50%)
  adding: kaggle/working/chfpozue/log/dispatcher.log (deflated 44%)
  adding: kaggle/working/chfpozue/checkpoint/ (stored 0%)
  adding: kaggle/working/chfpozue/trials/ (stored 0%)
  adding: kaggle/working/chfpozue/trials/OybzU/ (stored 0%)
  adding: kaggle/working/chfpozue/trials/OybzU/stderr (deflated 28%)
  adding: kaggle/working/chfpozue/trials/OybzU/.nni/ (stored 0%)
  adding: kaggle/working/chfpozue/trials/OybzU/.nni/state (stored 0%)
  adding: kaggle/working/chfpozue/trials/OybzU/.nni/metrics (deflated 73%)
  adding: kaggle/working/chfpozue/trials/OybzU/parameter.cfg (deflated 25%)
  adding: kaggle/working/chfpozue/trials/OybzU/stdout (def

In [13]:
# download NNI logs
from IPython.display import FileLink
FileLink(r'BOHBLogs.zip')

/kaggle/working/BOHBLogs.zip